In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('q_quora_train.csv')

C:\Users\harsh\AppData\Local\Temp\ipykernel_19944\2856031758.py:1: DtypeWarning: Columns (7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('q_quora_train.csv')


In [4]:
df.shape

(404351, 13)

In [5]:
df = df.iloc[:, :6]

In [7]:
df['is_duplicate'] = pd.to_numeric(df['is_duplicate'], errors='coerce')

In [8]:
df = df[df['is_duplicate'].isin([0, 1])]

In [26]:
df['is_duplicate'] = df['is_duplicate'].astype(int)

In [11]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [12]:
new_df = df.sample(30000)

In [13]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [14]:
new_df.duplicated().sum()

0

In [15]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
205950,Is interface theory of perception accepted in ...,Is Donald Hoffman’s interface theory of percep...
172191,Why does a pulsar [fastest rotating objects] c...,How can pulsars be seen when their beam is poi...
390529,How do people know she is the one?,How did you know she/he is the one?
364567,What would a Trump presidency mean for current...,How will Trump's presidency affect prospective...
309888,What is a good one-year anniversary gift for a...,What are some good first year anniversary gift...


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [17]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [18]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
205950,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
172191,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
390529,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
364567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
309888,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186017,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
318947,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
387255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
268382,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [20]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
205950,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
172191,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
390529,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
364567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
309888,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7396666666666667

In [24]:
!pip install xgboost

   ---------------------------------------- 0.0/56.8 MB ? eta -:--:--
    --------------------------------------- 0.8/56.8 MB 11.2 MB/s eta 0:00:06
    --------------------------------------- 1.0/56.8 MB 7.2 MB/s eta 0:00:08
   - -------------------------------------- 1.8/56.8 MB 3.9 MB/s eta 0:00:15
   -- ------------------------------------- 2.9/56.8 MB 4.0 MB/s eta 0:00:14
   -- ------------------------------------- 3.4/56.8 MB 3.9 MB/s eta 0:00:14
   --- ------------------------------------ 4.5/56.8 MB 3.9 MB/s eta 0:00:14
   --- ------------------------------------ 5.5/56.8 MB 3.9 MB/s eta 0:00:14
   ---- ----------------------------------- 6.3/56.8 MB 3.9 MB/s eta 0:00:13
   ---- ----------------------------------- 7.1/56.8 MB 3.9 MB/s eta 0:00:13
   ----- ---------------------------------- 7.9/56.8 MB 3.9 MB/s eta 0:00:13
   ------ --------------------------------- 8.7/56.8 MB 3.9 MB/s eta 0:00:13
   ------ --------------------------------- 9.4/56.8 MB 3.9 MB/s eta 0:00:13
   --

In [25]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7281666666666666